In [2]:

import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
import time



c:\Users\Jose\anaconda3\envs\ProyectoNegocios\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Cargar csv de datos limpiados
df = pd.read_csv('fra_cleaned.csv', encoding='latin-1', sep=';')

In [ ]:
def cargar_y_preparar_datos(filepath='fra_cleaned.csv'):
    
    try:
        df = pd.read_csv(filepath, encoding='latin1', sep=';')
        
    except FileNotFoundError:
        print(f"No se encontró '{filepath}'")
        
    
    return df

def crear_sopa_de_notas(df):
    
    # Lista de columnas que describen el olor
    columnas_descriptivas = [
        'Top', 'Middle', 'Base',
        'mainaccord1', 'mainaccord2', 'mainaccord3',
        'mainaccord4', 'mainaccord5',
        'Brand', 'Gender' 
    ]
    
    # Limpiar DF para evitar errore de concatenacion.
    df_limpio = df.copy()
    
    columnas_existentes = []
    for col in columnas_descriptivas:
        if col in df_limpio.columns:
            df_limpio[col] = df_limpio[col].fillna('')
            columnas_existentes.append(col)
        else:
            print(f"La columna '{col}' no se encontró.")
            
    columnas_descriptivas = columnas_existentes       

    #    Aplicar función join a cada fila para unir columnas en un string.
    def unir_fila(fila):
        return ' '.join([str(fila[col]) for col in columnas_descriptivas])

    df_limpio['perfil_olfativo'] = df_limpio.apply(unir_fila, axis=1)
    
    print("Ejemplos")
    print(df_limpio['perfil_olfativo'].iloc[0])
    print("."*50)
    print(df_limpio['perfil_olfativo'].iloc[1])
    print("."*50)
    
    return df_limpio

def generar_y_guardar_embeddings(df, model_name='all-MiniLM-L6-v2'):
    
    #Cargar modelo SBERT y generar embeddings
    
    
    # 1: Cargar modelo
    model = SentenceTransformer(model_name)

    # 2: Convertir en lista el string con la información olfativa.
    
    lista_perfiles = df['perfil_olfativo'].tolist()

    # 3: Generar Embeddings
    start_time = time.time()
    
    # El modelo vectoriza los strings de los perfiles olfativos.
    perfume_embeddings = model.encode(lista_perfiles, show_progress_bar=True)
    
    end_time = time.time()
    print(f"Tiempo: {end_time - start_time:.2f} segundos")
    print(f"Tamaño de la matriz de perfiles olfativos vectorizados: {perfume_embeddings.shape}")
    
    # 4: Guardar Embeddings
    # Se guarda el archivo en formato .npy para una carga más rápida.
    output_path = 'perfume_embeddings.npy'
    np.save(output_path, perfume_embeddings)
    

    # 5: Guardar índice de perfumes 
    # Sirve para identificar a que perfume corresponde cada fila de la matriz embedida.
    
    index_path = 'perfume_index.csv'
    df_index = df[['Perfume', 'Brand']].reset_index() # se guarda índice, nombre perfume y marca
    df_index.to_csv(index_path, index=False)
    
    

if __name__ == "__main__":
    # 1. Cargar csv
    df_perfumes = cargar_y_preparar_datos('fra_cleaned.csv')
    
    # 2. Crear sopa de notas
    df_con_sopa = crear_sopa_de_notas(df_perfumes)
    
    # 3. Generar y guardar embeddings
    if not df_con_sopa.empty:
        generar_y_guardar_embeddings(df_con_sopa)
    else:
        print("Error DF vacío")

Éxito: Se cargó 'fra_cleaned.csv' con 24063 filas.
Ejemplos:
fruity notes, aldehydes, green notes bulgarian rose, egyptian jasmine, lily-of-the-valley eucalyptus, pine rose woody fruity aromatic floral xerjoff unisex
..................................................
yuzu, citruses orange blossom, neroli musk, blonde woods citrus white floral sweet fresh musky jean-paul-gaultier women
..................................................


Batches:   0%|          | 2/752 [00:05<33:00,  2.64s/it]


KeyboardInterrupt: 